In [1]:
import torch
import random
import torch.nn as nn

In [2]:
%%capture

# download the files
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/dev.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/test.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/train.txt

# create the data folders
!mkdir data data/classes
!cp dev.txt data/classes
!cp test.txt data/classes
!cp train.txt data/classes

## Read and Process the Data

In [3]:
# function to read in data, process each line and split columns by " ||| "
def read_data(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.lower().strip()
            line = line.split(' ||| ')
            data.append(line)
    return data

train_data = read_data('data/classes/train.txt')
test_data = read_data('data/classes/test.txt')

# creating the word and tag indices
word_to_index = {}
word_to_index["<unk>"] = len(word_to_index) # add <UNK> to dictionary
tag_to_index = {}

# create word to index dictionary and tag to index dictionary from data
def create_dict(data, check_unk=False):
    for line in data:
        for word in line[1].split(" "):
            if check_unk == False:
                if word not in word_to_index:
                    word_to_index[word] = len(word_to_index)
            else:
                if word not in word_to_index:
                    word_to_index[word] = word_to_index["<unk>"]

        if line[0] not in tag_to_index:
            tag_to_index[line[0]] = len(tag_to_index)

create_dict(train_data)
create_dict(test_data, check_unk=True)

# create word and tag tensors from data
def create_tensor(data):
    for line in data:
        yield([word_to_index[word] for word in line[1].split(" ")], tag_to_index[line[0]])

train_data = list(create_tensor(train_data))
test_data = list(create_tensor(test_data))

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

## Model

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# create a simple neural network with embedding layer, bias, and xavier initialization
class DeepCBoW(nn.Module):
    def __init__(self, nwords, ntags, hidden_size, num_layers, emb_size):
        super(DeepCBoW, self).__init__()

        self.num_layers = num_layers

        # layers
        self.embedding = nn.Embedding(nwords, emb_size)
        self.linears = nn.ModuleList([nn.Linear(emb_size if i ==0 else hidden_size, hidden_size) \
            for i in range(num_layers)])

        # use xavier initialization for weights
        nn.init.xavier_uniform_(self.embedding.weight)
        for i in range(self.num_layers):
            nn.init.xavier_uniform_(self.linears[i].weight)

        # output layer
        self.output_layer = nn.Linear(hidden_size, ntags)

    def forward(self, x):
        emb = self.embedding(x) # seq x emb_size
        emb_sum = torch.sum(emb, dim=0) # emb_size
        h = emb_sum.view(1, -1) # reshape to (1, emb_size)
        for i in range(self.num_layers):
            h = torch.tanh(self.linears[i](h))
        out = self.output_layer(h) # 1 x ntags
        return out

HIDDEN_SIZE = 64
NUM_LAYERS = 2 # hidden layers
EMB_SIZE = 64
model = DeepCBoW(number_of_words, number_of_tags, HIDDEN_SIZE, NUM_LAYERS, EMB_SIZE).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
type = torch.LongTensor

if torch.cuda.is_available():
    model.to(device)
    type = torch.cuda.LongTensor

## Model Training

In [5]:
# perform training of the Bow model

for epoch in range(10):
    # perform training
    model.train()
    random.shuffle(train_data)
    total_loss = 0.0
    train_correct = 0
    for sentence, tag in train_data:
        sentence = torch.tensor(sentence).type(type)
        tag = torch.tensor([tag]).type(type)
        output = model(sentence)
        predicted = torch.argmax(output.data.detach()).item()

        loss = criterion(output, tag)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if predicted == tag: train_correct+=1

    # perform testing of the model
    model.eval()
    test_correct = 0
    for sentence, tag in test_data:
        sentence = torch.tensor(sentence).type(type)
        output = model(sentence)
        predicted = torch.argmax(output.data.detach()).item()
        if predicted == tag: test_correct += 1

    # print model performance results
    log = f'epoch: {epoch+1} | ' \
        f'train loss/sent: {total_loss/len(train_data):.4f} | ' \
        f'train accuracy: {train_correct/len(train_data):.4f} | ' \
        f'test accuracy: {test_correct/len(test_data):.4f}'

    print(log)

epoch: 1 | train loss/sent: 1.4265 | train accuracy: 0.3629 | test accuracy: 0.4285
epoch: 2 | train loss/sent: 1.0115 | train accuracy: 0.5886 | test accuracy: 0.3729
epoch: 3 | train loss/sent: 0.6532 | train accuracy: 0.7643 | test accuracy: 0.3787
epoch: 4 | train loss/sent: 0.4064 | train accuracy: 0.8590 | test accuracy: 0.3941
epoch: 5 | train loss/sent: 0.2661 | train accuracy: 0.9092 | test accuracy: 0.3805
epoch: 6 | train loss/sent: 0.1810 | train accuracy: 0.9386 | test accuracy: 0.3792
epoch: 7 | train loss/sent: 0.1266 | train accuracy: 0.9585 | test accuracy: 0.3765
epoch: 8 | train loss/sent: 0.0808 | train accuracy: 0.9721 | test accuracy: 0.3769
epoch: 9 | train loss/sent: 0.0604 | train accuracy: 0.9802 | test accuracy: 0.3778
epoch: 10 | train loss/sent: 0.0450 | train accuracy: 0.9830 | test accuracy: 0.3643
